In [ ]:
import os
import io
import imageio
import medmnist
import ipywidgets
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import glob
import pandas as pd
from matplotlib import pyplot as plt
import random 
import cv2 as cv
import math 
from keras.callbacks import EarlyStopping

# DATA
IMG_SIZE = 24
BATCH_SIZE = 32
FRAME = 10
CHANNEL = 1

AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, FRAME, CHANNEL)
NUM_CLASSES = 1

# OPTIMIZER
LEARNING_RATE = 1e-4

# TRAINING
EPOCHS = 100

MODEL_PATH = "model_h"
RESULT_PATH = "results_m"

In [2]:
np.random.seed(42)
random.seed(42)

test_flist = [] 
val_flist = [] 
train_flist = [] 

for cpath, dirs, files in os.walk('HUST-LEBW/training'):
    if '.jpg' in ''.join(files):
        train_flist += [ cpath + '/' + f for f in files if 'checkpoint' not in cpath]
            
for cpath, dirs, files in os.walk('HUST-LEBW/test'):
    if  '.jpg' in ''.join(files):
        val_flist += [ cpath + '/' + f for f in files if 'checkpoint' not in cpath]
        
for cpath, dirs, files in os.walk('MAEB'):
    if '.jpg' in ''.join(files):
        test_flist += [ cpath + '/' + f for f in files if 'checkpoint' not in cpath and np.sum(cv.imread(cpath + '/' + f)) !=0 ]
            
random.shuffle(train_flist)

test_flist = test_flist + val_flist


train_lb = [ 1 if f.split('/')[2] == 'blink' or f.split('/')[2] == 'close' or f.split('/')[3] == 'close' else 0 for f in train_flist ]
val_lb = [ 1 if f.split('/')[2] == 'blink' or f.split('/')[2] == 'close' or f.split('/')[3] == 'close' else 0 for f in val_flist ]
test_lb = [ 1 if f.split('/')[2] == 'blink' or f.split('/')[2] == 'close' or f.split('/')[3] == 'close' else 0 for f in test_flist ]

len(train_flist), len(val_flist),len(test_flist), len(train_lb), len(val_lb), len(test_lb)

(1723, 889, 20185, 1723, 889, 20185)

In [3]:
train_steps_per_epoch = math.ceil(len(train_flist) / BATCH_SIZE)
test_steps_per_epoch = math.ceil(len(test_flist) / BATCH_SIZE)
val_steps_per_epoch = math.ceil(len(val_flist) / BATCH_SIZE)

In [4]:
@tf.function
def get_img(path, mode='train', trans = [0, 2, 1, 3]):
    
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.rgb_to_grayscale(img)

    if mode =='train':
        img = tf.image.random_flip_left_right(img)
        
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.per_image_standardization(img)

    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE*FRAME))
    img = tf.reshape(img, (IMG_SIZE, FRAME,IMG_SIZE, CHANNEL))
    img = tf.transpose(img, trans)
    
    return img

def get_dataset(flist , lbs, mode = 'train', trans = [0 , 2, 1, 3]): # [1, 0, 2, 3]

    dataset = tf.data.Dataset.from_tensor_slices( flist )
    dataset_lb = tf.data.Dataset.from_tensor_slices( lbs )
    
    dataset = dataset.map(lambda x: get_img(x, mode, trans), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = tf.data.Dataset.zip((dataset, dataset_lb))

    if mode == 'train':
        dataset = dataset.repeat()

    dataset = dataset.batch(BATCH_SIZE)      
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

# 3Dcnn

In [5]:
def cnn3d(fid):
    
    train_dataset = get_dataset(train_flist, train_lb, mode = 'train', trans = [0 , 2, 1, 3])
    val_dataset = get_dataset(val_flist, val_lb, mode = 'test', trans = [0 , 2, 1, 3])
    test_dataset = get_dataset(test_flist, test_lb, mode = 'test', trans = [0 , 2, 1, 3])
    
    SEED = 42
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
    keras.utils.set_random_seed(SEED)
    random.seed(SEED)

    model = tf.keras.Sequential([
        tf.keras.layers.Conv3D(32, kernel_size=(3, 3, 3), activation='relu', 
                               padding = 'same',input_shape=(IMG_SIZE,IMG_SIZE,FRAME,CHANNEL)),
        tf.keras.layers.MaxPooling3D(),
        tf.keras.layers.Conv3D(32, 3, padding = 'same', activation='relu'),
        tf.keras.layers.MaxPooling3D(),
        tf.keras.layers.Conv3D(64, 3, padding = 'same', activation='relu'),
        tf.keras.layers.MaxPooling3D(),
        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1) 
    ])
    
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)

    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    model.fit(train_dataset, steps_per_epoch=train_steps_per_epoch, 
              epochs=EPOCHS, validation_data=val_dataset, validation_steps= val_steps_per_epoch, 
              callbacks=[tf.keras.callbacks.ModelCheckpoint(
            filepath='model/3DCNN_{}'.format(fid),save_best_only=True,
            monitor='val_accuracy',
            mode='max',)], verbose=1)

    best_model = keras.models.load_model('model/3DCNN_{}'.format(fid))

    preds = tf.nn.sigmoid(best_model.predict(test_dataset)).numpy().ravel()
    pd.DataFrame( np.array([test_flist, preds]).T, columns=['path', 'pred']).to_csv('{}/3DCNN_{}.csv'.format(RESULT_PATH, fid),index=False)

# CNN-LSTM

In [6]:
def cnn_():
    model = keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, 3, input_shape=(IMG_SIZE, IMG_SIZE, CHANNEL), padding='same', activation='relu'))    
    model.add(tf.keras.layers.MaxPool2D())

    model.add(tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPool2D())

    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu')) 
    model.add(tf.keras.layers.MaxPool2D())
    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu')) 
    model.add(tf.keras.layers.MaxPool2D())

    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.Dropout(0.5))
    #model.add(tf.keras.layers.GlobalAveragePooling2D())

    return model


def cnnlstm(fid):

    train_dataset = get_dataset(train_flist, train_lb, mode = 'train', trans = [1, 0, 2, 3])
    test_dataset = get_dataset(test_flist, test_lb, mode = 'test', trans =[1, 0, 2, 3])
    val_dataset = get_dataset(val_flist, val_lb, mode = 'test',  trans =[1, 0, 2, 3])

    SEED = 42
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
    keras.utils.set_random_seed(SEED)
    random.seed(SEED)

    model = keras.Sequential()
    convnet = cnn_()
    model.add(tf.keras.layers.TimeDistributed(convnet, input_shape=(FRAME,IMG_SIZE, IMG_SIZE, CHANNEL)))
    model.add(tf.keras.layers.LSTM(64))
    model.add(tf.keras.layers.Dense(1))

    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    model.fit(train_dataset, steps_per_epoch=train_steps_per_epoch, 
              epochs=EPOCHS, validation_data=val_dataset, validation_steps= val_steps_per_epoch, 
              callbacks=[tf.keras.callbacks.ModelCheckpoint(
            filepath='model/CNNLSTM_{}'.format(fid),save_best_only=True,
            monitor='val_accuracy',
            mode='max',)], verbose=1)

    best_model = keras.models.load_model('model/CNNLSTM_{}'.format(fid))
    
    preds = tf.nn.sigmoid(best_model.predict(test_dataset)).numpy().ravel()
    pd.DataFrame( np.array([test_flist, preds]).T, columns=['path', 'pred']).to_csv('{}/CNNLSTM_{}.csv'.format(RESULT_PATH, fid),index=False)

# Pyramid 3DCNN
### https://www.sciencedirect.com/science/article/pii/S0893608022001423

In [7]:
def P2B2(fid):
    # starting  block
    input_ = tf.keras.Input(shape=INPUT_SHAPE) 
    x = tf.keras.layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(1, 1, 2), padding = 'same')(input_)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling3D( pool_size=(3, 3, 3), strides=(1, 1, 2), padding = 'same')(x)
    
    
    ## pyramid 1
    # branch 1
    x1 = tf.keras.layers.Conv3D(64, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x)
    x1 = tf.keras.layers.BatchNormalization()(x1)

    # branch 2
    x2 = tf.keras.layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding = 'same')(x)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.Activation('relu')(x2)

    x2 = tf.keras.layers.Conv3D(64, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x2)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    
    
    #last = tf.keras.Sequential()   
    last = tf.keras.layers.Add()([x1,x2])   
    last = tf.keras.layers.Activation('relu')(last)
    
    
    
    ## pyramid 2
    # branch 1
    x1 = tf.keras.layers.Conv3D(128, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(last)
    x1 = tf.keras.layers.BatchNormalization()(x1)

    # branch 2
    x2 = tf.keras.layers.Conv3D(128, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding = 'same')(last)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.Activation('relu')(x2)

    x2 = tf.keras.layers.Conv3D(128, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x2)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    
    
    #last = tf.keras.Sequential()   
    last = tf.keras.layers.Add()([x1,x2])    
    last = tf.keras.layers.Activation('relu')(last)
    
    
    last = tf.keras.layers.GlobalAveragePooling3D()(last)
    last_out =tf.keras.layers.Dense(2, activation='softmax')(last)

    model = tf.keras.Model(inputs = input_,outputs = last_out)
    
    
    train_dataset = get_dataset(train_flist, train_lb, mode = 'train', trans = [0 , 2, 1, 3])
    test_dataset = get_dataset(test_flist, test_lb, mode = 'test', trans =[0 , 2, 1, 3])
    val_dataset = get_dataset(val_flist, val_lb, mode = 'test',   trans =[0 , 2, 1, 3])

    SEED = 42
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
    keras.utils.set_random_seed(SEED)
    random.seed(SEED)
    
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)

    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=["accuracy"],
    )

    model.fit(train_dataset, steps_per_epoch=train_steps_per_epoch, 
              epochs=EPOCHS, validation_data=val_dataset, validation_steps= val_steps_per_epoch, 
              callbacks=[tf.keras.callbacks.ModelCheckpoint(
            filepath='model/P2B2_{}'.format(fid),save_best_only=True,
            monitor='val_accuracy',
            mode='max',)], verbose=1)

    best_model = keras.models.load_model('model/P2B2_{}'.format(fid))


    preds = best_model.predict(test_dataset)[:,1]
    pd.DataFrame( np.array([test_flist, preds]).T, columns=['path', 'pred']).to_csv('{}/P2B2_{}.csv'.format(RESULT_PATH, fid),index=False)


In [8]:
def P2B3(fid):
    # starting  block
    input_ = tf.keras.Input(shape=INPUT_SHAPE) 
    x = tf.keras.layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(1, 1, 2), padding = 'same')(input_)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling3D( pool_size=(3, 3, 3), strides=(1, 1, 2), padding = 'same')(x)
    
    
    ## pyramid 1
    # branch 1
    x1 = tf.keras.layers.Conv3D(64, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x)
    x1 = tf.keras.layers.BatchNormalization()(x1)

    # branch 2
    x2 = tf.keras.layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding = 'same')(x)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.Activation('relu')(x2)

    x2 = tf.keras.layers.Conv3D(64, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x2)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    
    # branch 3
    x3 = tf.keras.layers.Conv3D(64, kernel_size=(5, 5, 3), strides=(1, 1, 1), padding = 'same')(x)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.Activation('relu')(x3)

    x3 = tf.keras.layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding = 'same')(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.Activation('relu')(x3)

    x3 = tf.keras.layers.Conv3D(64, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    
    #last = tf.keras.Sequential()   
    last = tf.keras.layers.Add()([x1,x2,x3])    
    last = tf.keras.layers.Activation('relu')(last)
    
    
    
    ## pyramid 2
    # branch 1
    x1 = tf.keras.layers.Conv3D(128, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(last)
    x1 = tf.keras.layers.BatchNormalization()(x1)

    # branch 2
    x2 = tf.keras.layers.Conv3D(128, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding = 'same')(last)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.Activation('relu')(x2)

    x2 = tf.keras.layers.Conv3D(128, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x2)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    
    # branch 3
    x3 = tf.keras.layers.Conv3D(128, kernel_size=(5, 5, 3), strides=(1, 1, 1), padding = 'same')(last)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.Activation('relu')(x3)

    x3 = tf.keras.layers.Conv3D(128, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding = 'same')(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.Activation('relu')(x3)

    x3 = tf.keras.layers.Conv3D(128, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    
    #last = tf.keras.Sequential()   
    last = tf.keras.layers.Add()([x1,x2,x3])    
    last = tf.keras.layers.Activation('relu')(last)
    
    
    last = tf.keras.layers.GlobalAveragePooling3D()(last)
    last_out =tf.keras.layers.Dense(2, activation='softmax')(last)

    model = tf.keras.Model(inputs = input_,outputs = last_out)
    
    
    train_dataset = get_dataset(train_flist, train_lb, mode = 'train', trans = [0 , 2, 1, 3])
    test_dataset = get_dataset(test_flist, test_lb, mode = 'test', trans =[0 , 2, 1, 3])
    val_dataset = get_dataset(val_flist, val_lb, mode = 'test',   trans =[0 , 2, 1, 3])

    SEED = 42
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
    keras.utils.set_random_seed(SEED)
    random.seed(SEED)
    
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath='{}/P2B3_{}'.format(MODEL_PATH , fid),save_best_only=True,
            monitor='val_loss',
            mode='min',)

    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=["accuracy"],
    )

    model.fit(train_dataset, steps_per_epoch=train_steps_per_epoch, 
              epochs=EPOCHS, validation_data=val_dataset, validation_steps= val_steps_per_epoch, 
              callbacks=[tf.keras.callbacks.ModelCheckpoint(
            filepath='model/P2B3_{}'.format(fid),save_best_only=True,
            monitor='val_accuracy',
            mode='max',)], verbose=1)

    best_model = keras.models.load_model('model/P2B3_{}'.format(fid))

    preds = best_model.predict(test_dataset)[:,1]
    pd.DataFrame( np.array([test_flist, preds]).T, columns=['path', 'pred']).to_csv('{}/P2B3_{}.csv'.format(RESULT_PATH, fid),index=False)


In [9]:
def P3B3(fid):
    # starting  block
    
    input_ = tf.keras.Input(shape=INPUT_SHAPE) 
    x = tf.keras.layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(1, 1, 2), padding = 'same')(input_)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 2), padding = 'same')(x)
    
    
    ## pyramid 1
    # branch 1
    x1 = tf.keras.layers.Conv3D(64, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x)
    x1 = tf.keras.layers.BatchNormalization()(x1)

    # branch 2
    x2 = tf.keras.layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding = 'same')(x)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.Activation('relu')(x2)

    x2 = tf.keras.layers.Conv3D(64, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x2)
    x2 = tf.keras.layers.BatchNormalization()(x2)

    # branch 3
    x3 = tf.keras.layers.Conv3D(64, kernel_size=(5, 5, 3), strides=(1, 1, 1), padding = 'same')(x)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.Activation('relu')(x3)

    x3 = tf.keras.layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding = 'same')(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.Activation('relu')(x3)

    x3 = tf.keras.layers.Conv3D(64, kernel_size=(1, 1, 3), strides=(2, 2, 1),  padding = 'same')(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)

    #last = tf.keras.Sequential()   
    last = tf.keras.layers.Add()([x1,x2,x3])    
    last = tf.keras.layers.Activation('relu')(last)
    
    
    
    ## pyramid 2
    # branch 1
    x1 = tf.keras.layers.Conv3D(128, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(last)
    x1 = tf.keras.layers.BatchNormalization()(x1)

    # branch 2
    x2 = tf.keras.layers.Conv3D(128, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding = 'same')(last)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.Activation('relu')(x2)

    x2 = tf.keras.layers.Conv3D(128, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x2)
    x2 = tf.keras.layers.BatchNormalization()(x2)

    # branch 3
    x3 = tf.keras.layers.Conv3D(128, kernel_size=(5, 5, 3), strides=(1, 1, 1), padding = 'same')(last)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.Activation('relu')(x3)

    x3 = tf.keras.layers.Conv3D(128, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding = 'same')(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.Activation('relu')(x3)

    x3 = tf.keras.layers.Conv3D(128, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)

    #last = tf.keras.Sequential()   
    last = tf.keras.layers.Add()([x1,x2,x3])    
    last = tf.keras.layers.Activation('relu')(last)
    
    
    
#     ## pyramid 3
    # branch 1
    x1 = tf.keras.layers.Conv3D(192, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(last)
    x1 = tf.keras.layers.BatchNormalization()(x1)

    # branch 2
    x2 = tf.keras.layers.Conv3D(192, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding = 'same')(last)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.Activation('relu')(x2)

    x2 = tf.keras.layers.Conv3D(192, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x2)
    x2 = tf.keras.layers.BatchNormalization()(x2)

    # branch 3
    x3 = tf.keras.layers.Conv3D(192, kernel_size=(5, 5, 3), strides=(1, 1, 1), padding = 'same')(last)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.Activation('relu')(x3)

    x3 = tf.keras.layers.Conv3D(192, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding = 'same')(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.Activation('relu')(x3)

    x3 = tf.keras.layers.Conv3D(192, kernel_size=(1, 1, 3), strides=(2, 2, 1), padding = 'same')(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)

    #last = tf.keras.Sequential()   
    last = tf.keras.layers.Add()([x1,x2,x3])    
    last = tf.keras.layers.Activation('relu')(last)
    
    last = tf.keras.layers.GlobalAveragePooling3D()(last)
    last_out = tf.keras.layers.Dense(2, activation='softmax')(last)

    model = tf.keras.Model(inputs = input_, outputs = last_out)
    
    
    train_dataset = get_dataset(train_flist, train_lb, mode = 'train', trans = [0 , 2, 1, 3])
    test_dataset = get_dataset(test_flist, test_lb, mode = 'test', trans =[0 , 2, 1, 3])
    val_dataset = get_dataset(val_flist, val_lb, mode = 'test',   trans =[0 , 2, 1, 3])

    SEED = 42
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
    keras.utils.set_random_seed(SEED)
    random.seed(SEED)
    
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=["accuracy"],
    )

    model.fit(train_dataset, steps_per_epoch=train_steps_per_epoch, 
              epochs=EPOCHS, validation_data=val_dataset, validation_steps= val_steps_per_epoch, 
              callbacks=[tf.keras.callbacks.ModelCheckpoint(
            filepath='model/P3B3_{}'.format(fid),save_best_only=True,
            monitor='val_accuracy',
            mode='max',)], verbose=1)

    best_model = keras.models.load_model('model/P3B3_{}'.format(fid))

    preds = best_model.predict(test_dataset)[:,1]
    pd.DataFrame( np.array([test_flist, preds]).T, columns=['path', 'pred']).to_csv('{}/P3B3_{}.csv'.format(RESULT_PATH, fid),index=False)


## Train

In [ ]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

# cross validation
for fid in range(1,6):
    cnnlstm(fid)


for fid in range(1,6):
    cnn3d(fid)

for fid in range(1,6):
    P2B2(fid)

for fid in range(1,6):
    P2B3(fid)

